In [1]:
import pandas as pd
import csv
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
# reading the CSV file
passengers = pd.read_csv('train.csv')
 
# displaying the contents of the CSV file
# print(passengers.columns)
passengers = passengers[['PassengerId', 'Pclass', 
'Name', 'Sex', 'Age', 'SibSp','Parch', 'Ticket', 
'Fare', 'Cabin', 'Embarked','Survived']]

passengers = passengers.drop(columns=['PassengerId','Name', 'Ticket','Cabin'])

print(passengers)

     Pclass     Sex   Age  SibSp  Parch     Fare Embarked  Survived
0         3    male  22.0      1      0   7.2500        S         0
1         1  female  38.0      1      0  71.2833        C         1
2         3  female  26.0      0      0   7.9250        S         1
3         1  female  35.0      1      0  53.1000        S         1
4         3    male  35.0      0      0   8.0500        S         0
..      ...     ...   ...    ...    ...      ...      ...       ...
886       2    male  27.0      0      0  13.0000        S         0
887       1  female  19.0      0      0  30.0000        S         1
888       3  female   NaN      1      2  23.4500        S         0
889       1    male  26.0      0      0  30.0000        C         1
890       3    male  32.0      0      0   7.7500        Q         0

[891 rows x 8 columns]


In [4]:
X_train = passengers.iloc[:,0:-1]
y_train = passengers.iloc[:,-1]
categorical_cols = ['Sex','Pclass','Embarked']
#import pandas as pd
X_train = pd.get_dummies(X_train, columns = categorical_cols)
print(np.mean(X_train['Age']))

for i in range(len(X_train['Age'])):
    if np.isnan(X_train['Age'][i]):
        X_train['Age'][i] = np.mean(X_train['Age'])
X_train['Age']
        

29.69911764705882


/var/folders/bj/xb0vw9_96cx1kj56y4l_zzww0000gn/T/ipykernel_25348/2521894574.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Age'][i] = np.mean(X_train['Age'])


0      22.000000
1      38.000000
2      26.000000
3      35.000000
4      35.000000
         ...    
886    27.000000
887    19.000000
888    29.699118
889    26.000000
890    32.000000
Name: Age, Length: 891, dtype: float64

In [5]:
xg_class = xgb.XGBClassifier(objective ='reg:linear', colsample_bytree = 0.2, learning_rate = 0.0000007,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_class.fit(X_train,y_train)

preds_train = xg_class.predict(X_train)
sum(preds_train == y_train) / len(y_train)

[13:53:02] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-37/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


0.7295173961840629

In [17]:
mlp_class = MLPClassifier( learning_rate='adaptive', max_iter=1000, hidden_layer_sizes=(64, 64))
mlp_class.fit(X_train,y_train)
print(round(mlp_class.score(X_train,y_train) * 100,2), '%')


80.58 %


In [7]:
#### TEST SET
# reading the CSV file
test = pd.read_csv('test.csv')
pID = test.iloc[:,0]
test = test.drop(columns=['PassengerId', 'Name', 'Ticket','Cabin'])
test = pd.get_dummies(test, columns = categorical_cols)
# print(test.info)

for i in range(len(test['Age'])):
    if np.isnan(test['Age'][i]):
        test['Age'][i] = np.mean(test['Age'])
    if np.isnan(test['Fare'][i]):
        test['Fare'][i] = np.mean(test['Fare'])


preds_test = mlp_class.predict(test)


/var/folders/bj/xb0vw9_96cx1kj56y4l_zzww0000gn/T/ipykernel_25348/581079917.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Age'][i] = np.mean(test['Age'])
/var/folders/bj/xb0vw9_96cx1kj56y4l_zzww0000gn/T/ipykernel_25348/581079917.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Fare'][i] = np.mean(test['Fare'])


In [8]:
submission = np.column_stack((np.array(pID), np.array(preds_test)))
submission = np.row_stack((['PassengerId','Survived'],submission))
with open('submission.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    for i in range(len(submission)):
        writer.writerow(submission[i])




In [9]:
DTC = DecisionTreeClassifier()
DTC.fit(X_train,y_train)
DTC.score(X_train,y_train)
preds_test = DTC.predict(test)

In [10]:
print("preds_test: ", preds_test)

preds_test:  [0 0 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 0 0 0 1 0 1 1 0
 0 0 1 1 0 0 1 1 0 1 0 1 1 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 0 1 0 0
 0 1 1 1 0 0 1 0 1 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0
 1 1 1 0 0 0 0 0 0 1 0 0 1 0 1 1 1 1 1 1 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 1 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1 1 1 1 1 0 1 1 1
 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0
 0 0 1 0 1 0 0 1 0 0 0]
